In [ ]:
!pip install qiskit==0.7.2
!pip install pylatexenc
!pip install qiskit-machine-learning

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.0/223.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 4.4 MB/s eta 0:00:00
  Command arguments: /usr/bin/python3 -u -c '
  exec(compile('"'"''"'"''"'"'
  # This is <pip-setuptools-caller> -- a caller that pip uses to run setup.py
  #
  # - It imports setuptools before invoking setup.py, to enable projects that directly
  #   import from `distutils.core` to work with newer packaging standards.
  # - It provides a clear error message when setuptools is not installed.
  # - It sets `sys.argv[0]` to the underlying `setup.py`, when invoking `setup.py` so

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from qiskit.circuit.library import PauliFeatureMap, EfficientSU2,RealAmplitudes,ZZFeatureMap
from qiskit_algorithms.optimizers import SPSA,COBYLA, ADAM
from qiskit.primitives import Sampler
from qiskit.circuit.library import EfficientSU2
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from qiskit.circuit.library import ZFeatureMap
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

In [ ]:
class VarationalQuantumClassifier:
   def __init__(self) -> None:
        self.data_path = "/content/oito_sitios (1).csv"
        annni_dataset = pd.read_csv(self.data_path)
        self.X = annni_dataset[['feature_5','feature_6','feature_7','feature_9','feature_10','feature_11']]
        self.y = annni_dataset['label'] -1
        self.y = self.y.to_numpy().reshape(-1, 1)
        self.g = annni_dataset['g']
        self.k = annni_dataset['k']

   def process_data(self):
        X_train, X_test, y_train, y_test = train_test_split(self.X , self.y, test_size=0.3, random_state=44)
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        return X_train, X_test, y_train, y_test  # Retorna y_train e y_test sem codificação one-hot

   def model_and_score(self):
        X_train, X_test, y_train, y_test = self.process_data()
        feature_map = ZFeatureMap(feature_dimension=6, reps=3)
        otimizador = SPSA(maxiter=100)
        sampler = Sampler(options={'shots':600})
        ansatz = EfficientSU2(6, reps=5, entanglement='linear', insert_barriers=True)
        vqc = VQC(sampler = sampler,feature_map=feature_map ,ansatz=ansatz,optimizer = otimizador)
        vqc.fit(X_train,y_train)
        predictions = vqc.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        print(accuracy)
        return vqc

   def plot_graph(self,modelo):
        scaler = MinMaxScaler()
        X = scaler.fit_transform(self.X)
        predict = modelo.predict(X)
        plt.scatter( self.g,self.k, c=predict)
        plt.title('Model of ANNNI with VQC')
        plt.xlabel('k')
        plt.ylabel('g')
        plt.grid()

**Usando o normalize**

In [ ]:
class VarationalQuantumClassifier:
   def __init__(self) -> None:
        self.data_path = "/content/sample_balanced_oito_sitios_label.csv"
        annni_dataset = pd.read_csv(self.data_path)
        self.X = annni_dataset[['feature_5','feature_6','feature_7','feature_9','feature_10','feature_11']]
        self.y = annni_dataset['label'] -1
        self.y = self.y.to_numpy().reshape(-1, 1)
        self.g = annni_dataset['g']
        self.k = annni_dataset['k']

   def process_data(self):
        X_train, X_test, y_train, y_test = train_test_split(self.X , self.y, test_size=0.3, random_state=44)
        scaler = MinMaxScaler()
        X_train_normalized = normalize(X_train, norm='l2')
        X_test_normalized = normalize(X_test, norm='l2')

        return X_train_normalized, X_test_normalized, y_train, y_test  # Retorna y_train e y_test sem codificação one-hot

   def model_and_score(self):
        X_train, X_test, y_train, y_test = self.process_data()
        feature_map = ZFeatureMap(feature_dimension=6, reps=3)
        otimizador = SPSA(maxiter=100)
        sampler = Sampler(options={'shots':600})
        ansatz = EfficientSU2(6, reps=5, entanglement='linear', insert_barriers=True)
        vqc = VQC(sampler = sampler,feature_map=feature_map ,ansatz=ansatz,optimizer = otimizador)
        vqc.fit(X_train,y_train)
        predictions = vqc.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        print(accuracy)
        return vqc

   def plot_graph(self,modelo):
        X = normalize(X, norm='l2')
        predict = modelo.predict(X)
        plt.scatter( self.g,self.k, c=predict)
        plt.title('Model of ANNNI with VQC')
        plt.xlabel('k')
        plt.ylabel('g')
        plt.grid()


model = VarationalQuantumClassifier()
modelo = model.model_and_score()